In [36]:
from gensim import corpora
import numpy as np
import unicodecsv as csv
import tensorflow as tf
import math
import os,sys
import random
from scipy.sparse import csr_matrix
from tqdm import tqdm
import networkx as nx
import json

In [16]:
tsv = csv.reader(file('../data/MajorPapers.txt'), delimiter = '\t')
dbpid2pid={}
#pid2dbpid={}
#pid2title={}
pid=0
for row in tsv:
    #row[0] pid 
    #row[1] title
    dbpid=row[0]
    dbpid2pid[dbpid]=pid
    #pid2title[pid]=nltk.word_tokenize(row[1])
    pid+=1

print "The number of papers:%d"%len(dbpid2pid)

#authors
tsv = csv.reader(file('../data/MajorAuthors.txt'), delimiter = '\t')
dbaid2aid={}
aid2aname={}
aid=0
for row in tsv:
    #row[0] dbaid
    #row[1] author name
    dbaid=row[0]
    aname=row[1]
    dbaid2aid[dbaid]=aid
    aid2aname[aid]=aname
    aid+=1

#author-paper
tsv = csv.reader(file('../data/MajorPaperAuthor.txt'), delimiter = '\t')
aid2pids={}
#iitialize aid2pids
for aid in aid2aname:
    aid2pids[aid]=[]
#collect aids
for row in tsv:
    #row[0] dbpid 
    #row[1] dbaid
    dbpid=row[0]
    pid=dbpid2pid[dbpid]
    dbaid=row[1]
    aid=dbaid2aid[dbaid]
    aid2pids[aid].append(pid)

author_paper_indices=[]
author_paper_values=[]
author_paper_shape=(len(aid2aname), len(dbpid2pid))
for aid in aid2pids:
    for pid in aid2pids[aid]:
        author_paper_indices.append([aid,pid])
        author_paper_values.append(1)
indeces=np.array(author_paper_indices).T
author_paper=csr_matrix((author_paper_values, indeces), shape=author_paper_shape, dtype=np.int32)

print "The number of authors:%d"%author_paper.shape[0]

The number of papers:88675
The number of authors:102587


In [3]:
co_author_matrix=np.dot(author_paper,author_paper.T)
for i in xrange(co_author_matrix.shape[0]):
    co_author_matrix[i,i]=0
co_author_matrix.eliminate_zeros()

In [26]:
#print dbaid2aid["80CF3CF9"]Ying
print aid2aname[71929]
print aid2aname[70282]
#print co_author_matrix[71929]
print co_author_matrix[70282,70282]
print co_author_matrix[71929,70282]
print co_author_matrix[70282,71929]

ying ding
jie tang
0
7
7


In [29]:
G = nx.from_scipy_sparse_matrix(co_author_matrix)
G_max=max(nx.connected_component_subgraphs(G), key=len)
new_co_author_matrix=nx.adjacency_matrix(G_max)

In [31]:
new_aid2old_aid={}
new_aid2name={}
for new_aid,old_aid in enumerate(G_max.nodes()):#aidがそのまま残っている。インデックスとしてはバラバラ
    new_aid2old_aid[new_aid]=old_aid
    new_aid2name[new_aid]=aid2aname[old_aid]

In [43]:
new_aid2dabid={}
aid2dbaid=dict((v, k) for k, v in dbaid2aid.iteritems())
for new_aid,old_aid in new_aid2old_aid.iteritems():
    dbaid=aid2dbaid[old_aid]
    new_aid2dabid[new_aid]=dbaid

In [38]:
#ref http://stackoverflow.com/questions/8955448/save-load-scipy-sparse-csr-matrix-in-portable-data-format

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [40]:
save_sparse_csr("../data/co-author-matrix.npz",new_co_author_matrix)
with open('../data/co-author-index.json', 'w') as f:
    json.dump(new_aid2name, f, sort_keys=True, indent=4)

In [44]:
with open('../data/co-author-original-ids.json', 'w') as f:
    json.dump(new_aid2dabid, f, sort_keys=True, indent=4)

In [41]:
new_co_author_matrix.shape

(74530, 74530)